In [57]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Overview

This is a quick start on gertting reports with the Deep Research API on Agentspace. Full documentation in the [Official Docs](https://cloud.google.com/agentspace/agentspace-enterprise/docs/research-assistant)

## Checklist before you start


*   Make sure the agent builder service is already activated in the project. If it is not, you may have trouble getting agentspace configured
Make sure someone has the appropriate level [Role](https://cloud.google.com/agentspace/agentspace-enterprise/docs/before-you-begin#access-control)

*   Make sure (as a precaution) there is a at least one GCS bucket with read write.
*   Strongly recommend to make it easier to create a minimal datastore. In a crunch, you can create a datastore off of an empty google site source
*   Make sure there are no policy restrictions on creating new resources (agentbuilder). Many firms have policies that restrict the creation of new resources or services, if you encounter this work with the org policy admin staff.
*   Follow guidance at AgentSpace Enterprise public docs
  *   [Google Agentspace Enterprise overview](https://cloud.google.com/agentspace/agentspace-enterprise/docs/overview#start)
  *  [Get started with Agentspace Enterprise](https://cloud.google.com/agentspace/agentspace-enterprise/docs/quickstart-agentspace#before-you-begin)



## Getting Started

### Install Google Gen AI SDK for Python

In [58]:
%pip install --upgrade --quiet google-genai httpx json-stream

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set up Google Cloud Project

You'll need to set up authentication by choosing **one** of the following methods:

1.  **Use a Google Cloud Project:** Recommended for most users, this requires enabling the Vertex AI API in your Google Cloud project.
    - [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)
    - Run the cell below to set your project ID and location.
    - Read more about [Supported locations](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/locations)
This tutorial uses a Google Cloud Project for authentication.

In [ ]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))




### Set up your agentspace app

You will obtain the agentspace app id from the UI after creating an Agentspace [app](https://cloud.google.com/agentspace/agentspace-enterprise/docs/create-app)

In [ ]:

AS_APP_ID = "[your-as-app-id]]"  # @param {type: "string", placeholder: "[your-as-app-id]", isTemplate: true}
# leave as "default_assistant" if the app is created from the console UI
AS_ASSISTANT_ID = "default_assistant" # @param {type: "string", placeholder: "default_assistant", isTemplate: true}

### Import libraries

In [100]:
import requests
import httpx
from IPython.display import display, Markdown,HTML,Image
import google.auth.transport.requests
import json
import json_stream.httpx
import re
import time
from typing import List, Dict, Any, Tuple
import markdown

### Setup re-usable variables

In [162]:
# Define the base URL for the Google Discovery Engine API (Agentspace).
# This is the root endpoint for all API calls.
baseurl = 'https://discoveryengine.googleapis.com/v1alpha'

# Authenticate and create a bearer token for API requests.
# This uses Google's default application credentials.
creds, _ = google.auth.default()
creds.refresh(google.auth.transport.requests.Request())

# Set up the authorization headers required for the API call.
# This includes the Bearer token and the user project ID.
headers = {
    'Authorization': f'Bearer {creds.token}',
    'Content-Type': 'application/json',
    'X-Goog-User-Project': PROJECT_ID
}

# Construct the full request URL for the streamAssist endpoint.
# This endpoint is used to interact with a specific assistant.
request_url = f"{baseurl}/projects/{PROJECT_ID}/locations/global/collections/default_collection/engines/{AS_APP_ID}/assistants/{AS_ASSISTANT_ID}:streamAssist"


### Formatting functions
Functions to format markdown output from the API

In [158]:
def extract_and_format_research_plan(json_data_string: str) -> str:
    """
    Parses a JSON string to find a research plan, then extracts and formats
    it into a user-friendly Markdown numbered list.

    Args:
        json_data_string: A string containing the JSON data.

    Returns:
        A string containing the formatted research plan in Markdown,
        or an error message if the plan cannot be found.
    """
    try:
        # Step 1: Load the JSON string into a Python object
        data = json.loads(json_data_string)

        # Step 2: Navigate the nested structure to find the research plan
        # The plan is identified by a specific 'contentKind' metadata tag.
        research_plan_text = None
        # Safely access nested keys using .get() to avoid errors if a key is missing
        replies = data[0].get('answer', {}).get('replies', [])

        for reply in replies:
            content_metadata = reply.get('groundedContent', {}).get('contentMetadata', {})
            if content_metadata.get('contentKind') == 'RESEARCH_PLAN':
                # Found it! Extract the text content.
                research_plan_text = reply.get('groundedContent', {}).get('content', {}).get('text')
                break

        if not research_plan_text:
            return "Could not find a research plan in the provided JSON."

        # Step 3: Process the extracted text to create a clean Markdown output
        # The text is split into logical blocks based on double newlines.
        blocks = research_plan_text.strip().split('\n\n')

        # The first block is the main title of the research plan.
        title = blocks[0]

        # The subsequent blocks are the individual research steps.
        step_blocks = blocks[1:]

        # Step 4: Build the final Markdown string
        markdown_output = [f"{title}\n"]
        markdown_output.append("Here are the research steps:")

        for i, block in enumerate(step_blocks):
            # Each block contains a sub-heading and the main question.
            # We'll clean them up and combine them into a single, clear step.
            lines = [line.strip() for line in block.split('\n') if line.strip()]

            # The first line is the sub-heading (e.g., "### Understanding the Recent Changes")
            # We remove the Markdown heading characters '###'
            sub_heading = lines[0].replace('###', '').strip()

            # The second line is the descriptive question for the step.
            description = lines[1]

            # Combine them into a single, bolded list item.
            formatted_step = f"{i + 1}. **{sub_heading}:** {description}"
            markdown_output.append(formatted_step)

        return "\n".join(markdown_output)

    except (json.JSONDecodeError, IndexError, KeyError) as e:
        return f"An error occurred while processing the JSON: {e}"

def format_report_as_markdown(segments: List[Dict], refs: Dict[int, Dict]) -> str:
    """
    Formats the final list of segments into a clean Markdown document
    with a single, consolidated reference list at the end.
    """
    markdown_lines = []
    link_definitions = []
    citation_map: Dict[int, int] = {}
    citation_counter = 1

    # Process each text segment to add footnote markers
    for segment in segments:
        text = segment.get('text', '')
        ref_indices = segment.get('refs', []) # This list should now be populated

        citation_markers = ""
        for ref_index in ref_indices:
            if ref_index not in citation_map:
                citation_map[ref_index] = citation_counter
                citation_counter += 1

            footnote_num = citation_map[ref_index]
            citation_markers += f" [^{footnote_num}]"

        markdown_lines.append(text + citation_markers)

    # Build the list of link definitions for the footnotes
    if citation_map:
        sorted_citations = sorted(citation_map.items(), key=lambda item: item[1])

        for ref_index, footnote_num in sorted_citations:
            ref_data = refs.get(ref_index, {})
            title = ref_data.get('title', 'No Title')
            uri = ref_data.get('uri', '#')
            link_definitions.append(f"[^{footnote_num}]: [{title}]({uri})")

    # Combine the content and the footnote definitions
    main_content = "\n".join(markdown_lines)
    references_section = "\n".join(link_definitions)

    return f"{main_content}\n\n---\n\n### References\n{references_section}"


def format_segments_to_html(segments: List[Dict], refs: Dict[int, Dict]) -> str:
    """Generic function to format any list of segments into HTML."""
    html_lines = []
    style = (
        "style=\""
        "display: inline-block; background-color: #f0f0f0; border-radius: 12px; "
        "padding: 2px 10px; font-size: 12px; font-family: sans-serif; "
        "color: #555; text-decoration: none; margin-left: 6px; border: 1px solid #ccc;\""
    )

    for segment in segments:
        text = segment.get('text', '')
        ref_indices = segment.get('refs', []) # This will now be correctly populated

        citation_html = ""
        for ref_index in ref_indices:
            ref_data = refs.get(ref_index)
            if ref_data:
                uri = ref_data.get('uri', '#')
                title = ref_data.get('title', 'No Title')
                short_title = ' '.join(title.split()[:3]) + '...'
                citation_html += f' <a href="{uri}" {style} title="{title}" target="_blank">{short_title}</a>'

        text_with_citations = text + citation_html
        html_lines.append(markdown.markdown(text_with_citations))

    return "\n".join(html_lines)

### Define your search query

In [163]:
SEARCH_QUERY = "what is the latest in the semi-conductor industry"  # @param {type: "string", placeholder: "[your-search-query]", isTemplate: true}


### Step 1  Call to create a plan (save session info)


In [164]:


def create_bp_job(search_query:str):
  """
  Sends an initial query to the Agentspace assistant to start a research task.

  Args:
    search_query (str): The initial research query or topic.

  Returns:
    requests.Response: The HTTP response object from the API call.
  """
  post_data= {
         "query": {
           "text": f"{search_query}"
           },
         "answer_generation_mode": "research",
         "google_search_grounding_enabled":"TRUE",
         "userMetadata":{"timeZone":"America/New_York"},
         #"assistSkippingMode":"REQUEST_ASSIST",
         }

  # Send the POST request to the API endpoint.
  # The payload is converted to a JSON string.
  result = requests.post(
      url=request_url,
      data=json.dumps(post_data),
      headers=headers
  )

  # Return the entire response object.
  return result

# --- Execution and Session Handling ---

# Define the initial search query.
search_query = SEARCH_QUERY

# Call the function to create the initial research job.
initial_result = create_bp_job(search_query)
# Display the streamed response text in a Markdown format for better readability.
# The API returns a stream of JSON objects, so this will show the raw text.
research_plan = extract_and_format_research_plan(initial_result.text)
display(Markdown(research_plan))

# The response is a list of JSON objects. We parse the first one to get session info.
# This is crucial for maintaining the conversation context.
response_json = initial_result.json()
session_info = response_json[0]['sessionInfo']
# Extract the unique session identifier.
session_id = session_info['session']

## Semiconductor Industry Research Plan

Here are the research steps:
1. ****Overview of the Semiconductor Industry**:** What is the current state of the semiconductor industry, including market size, key players, and major trends?
2. ****Recent Technological Advancements**:** What are the latest breakthroughs in semiconductor technology, such as new materials, manufacturing processes, or chip architectures?
3. ****Market Trends and Demand Drivers**:** What are the current market trends driving demand for semiconductors, such as AI, automotive, or consumer electronics?
4. ****Supply Chain Issues and Geopolitical Factors**:** What are the current challenges and disruptions in the semiconductor supply chain, including geopolitical risks and trade tensions?
5. ****New Semiconductor Companies and Startups**:** What new companies or startups are emerging in the semiconductor industry, and what innovative technologies are they developing?
6. ****Future Outlook and Predictions**:** What are the expert predictions and forecasts for the future of the semiconductor industry, including potential growth areas and challenges?

### Step 2  Call with a payload holding session_info

In [ ]:
 #--- Follow-up Interaction ---

# Define the follow-up query to instruct the assistant to start the plan.
follow_up_query = 'Start research'

# Create the payload for the follow-up request.
# This payload is similar to the first one but crucially includes the session ID.
q= 'Start research'
run_plan_json ={
         "query": {
           "text": f"{q}"
           },
         "answer_generation_mode": "research",
         "google_search_grounding_enabled":"TRUE",
         "userMetadata":{"timeZone":"America/New_York"},
         "session":f"{session_id}",
         }

# --- Global Buffers and State Flags ---
qa_segments: List[Dict[str, Any]] = []
report_segments: List[Dict[str, Any]] = []
references: Dict[int, Dict[str, str]] = {}
current_context: str = None

# Temporary state-tracking variables
text_buffer: str = ''
current_segment_data: Dict[str, Any] = {}
last_segment_index: int = -1

def visitor(item: Any, path: Tuple):
    """
    Processes the stream, prints live updates for all content types,
    and correctly populates data structures for final formatting.
    """
    global text_buffer, current_context, qa_segments, report_segments, references
    global current_segment_data, last_segment_index

    # Capture text into a temporary buffer
    if path and path[-1] == 'text':
        text_buffer = item.strip()

  # Part 1: Capture all reference details (URI and Title)
    if 'references' in path:
        try:
            ref_index = int(path[path.index('references') + 1])
            if ref_index not in references: references[ref_index] = {}
            if path[-1] == 'uri': references[ref_index]['uri'] = item
            elif path[-1] == 'title': references[ref_index]['title'] = item
        except (ValueError, IndexError): pass

    # Part 2: Set the current context (answer vs. report)
    if 'contentKind' in path and path[-2] == 'contentMetadata':
        if item == 'RESEARCH_ANSWER': current_context = 'answer'
        elif item == 'RESEARCH_REPORT': current_context = 'report'

  # Part 3: Capture report segments with their references
    if 'segments' in path:
        try:
            segment_index = int(path[path.index('segments') + 1])

            # When the segment index changes, save the completed segment data
            if segment_index != last_segment_index and current_segment_data:
                if 'text' in current_segment_data:
                    if current_context == 'answer': qa_segments.append(current_segment_data)
                    elif current_context == 'report': report_segments.append(current_segment_data)
                current_segment_data = {} # Reset for the new segment

            last_segment_index = segment_index
            # Check if the path indicates this item is a segment's text
            if path[-1] == 'text':
                current_segment_data['text'] = item
                print(f"  -> Segment Received: of type {current_context}: {item[:100]}...")
            # Check if this item is a reference number within a 'referenceIndices' list
            elif 'referenceIndices' in path:
                # Initialize the 'refs' key as a list if it's not already there
                if 'refs' not in current_segment_data:
                    current_segment_data['refs'] = []
                # Append the reference number (item) to the list
                current_segment_data['refs'].append(int(item))
        except (ValueError, IndexError): pass

def finalize_stream():
    """Saves the last processed segment to the correct list."""
    if current_segment_data and 'text' in current_segment_data:
        if current_context == 'answer': qa_segments.append(current_segment_data)
        elif current_context == 'report': report_segments.append(current_segment_data)

# Start the Research. This is just a POST to the same URL
with httpx.Client() as client, client.stream("POST",url=request_url,data=json.dumps(run_plan_json),headers=headers,timeout=90) as r:
  try:
    json_stream.httpx.visit(r, visitor)
  finally:
    finalize_stream()


  -> Segment Received: of type answer: The semiconductor industry's market size is currently estimated to be between USD 627 billion and US...
  -> Segment Received: of type answer: *   **2024:** The market was valued at USD 681.05 billion. ...
  -> Segment Received: of type answer: Other reports indicate values of USD 584.17 billion and USD 694.0 billion.
...
  -> Segment Received: of type answer: *   **2025:** Projections range from USD 627.76 billion to USD 755.28 billion. ...
  -> Segment Received: of type answer: One source anticipates USD 702.44 billion. ...
  -> Segment Received: of type answer: Another forecasts USD 697 billion.
...
  -> Segment Received: of type answer: *   **2030:** The market is predicted to reach USD 950.97 billion.
...
  -> Segment Received: of type answer: *   **2032:** It is projected to reach USD 2,062.59 billion.
...
  -> Segment Received: of type answer: *   **2033:** One source estimates a market size of USD 1,221.24 billion.
...
  -> Segment Receive

### Step 3 Display the final report

#### Version 1: Markdown

In [159]:
# Generate the final Markdown report
final_markdown = format_report_as_markdown(report_segments, references)


print("\n\n\n================ FINAL MARKDOWN REPORT ================")
display(Markdown(final_markdown))




================ FINAL MARKDOWN REPORT ================


*   **A difficult path to rejoining:** A future U.S. president wishing to rejoin the Paris Agreement would have to submit a new NDC that is more ambitious than the last, which could be challenging given potential rollbacks of environmental protections.

## Research Report: Ramifications of Recent Changes in Paris Climate Agreement Membership

This report details the ramifications of recent changes in the Paris Climate Agreement membership, focusing primarily on the United States' withdrawals and rejoining, as per the provided research plan.

### **Understanding the Recent Changes in Membership**

The Paris Agreement is a landmark international treaty adopted in 2015 at the United Nations Climate Change Conference near Paris, France [^1]
. It was negotiated by 196 parties and signed in 2016 [^1]
. The agreement officially entered into force on November 4, 2016 [^1] [^2] [^3] [^4]
. Its core objective is to combat climate change by limiting the global temperature increase this century to well below 2 degrees Celsius above pre-industrial levels, while pursuing efforts to limit the increase to 1.5 degrees Celsius [^5] [^2]
. It also focuses on climate change mitigation, adaptation, and finance [^1]
.

As of February 2023, 195 members of the United Nations Framework Convention on Climate Change (UNFCCC) were parties to the agreement [^1] [^2]
. This number was later updated to 194 states and the European Union (EU) having ratified or acceded to the agreement, representing over 98% of global greenhouse gas emissions [^6]
. All 198 UNFCCC members have either signed or acceded to the Paris Agreement [^6]
. Key players like China and the United States, the countries with the first and second largest CO2 emissions among UNFCCC members, have ratified the agreement [^6]
.

**Recent Changes:**

*   **United States:** The United States is the only country to have formally withdrawn from the agreement [^7]
.
    *   President Donald Trump initiated the withdrawal process in 2020, and it became effective in November 2020 [^7] [^1] [^8]
.
    *   President Joe Biden officially rejoined the agreement in 2021 [^1]
.
    *   President Trump signed an Executive Order in 2025 to withdraw the United States from the Paris Agreement for a second time [^7] [^9] [^10]
. This withdrawal is expected to take effect sometime in January 2026, one year after official notification was submitted to the UN [^11] [^12] [^13] [^14]
.
*   **Countries Not Formally Approved:** As of February 2023, Iran, Libya, and Yemen had not formally approved the agreement [^7]
. Of these, Iran is the only major emitter [^1]
. As of March 11, 2025, these countries, along with the U.S., were the only ones not committed [^15]
.

### **Impact on Global Emissions Targets**

The Paris Agreement aims for a shift to net-zero emissions by 2050 to limit temperature rise to 1.5°C [^16]
. However, current data suggests these goals are not being met [^16]
. The agreement's aggressive target depends heavily on controlling fossil fuel use, especially in transportation [^17]
. Withdrawal from the agreement, particularly by major economies, and anti-environmental actions can significantly hinder progress [^17]
.

**Impact of US Withdrawal (2020):**

*   The withdrawal made it significantly more difficult to maintain or enhance emission reduction efforts globally [^12]
.
*   It created a leadership vacuum, although other nations, state and local leaders, and businesses partially filled this void [^12]
.
*   The global climate finance goal of $1.3 trillion per year by 2035 became much harder to achieve due to the U.S. halting contributions [^12]
.
*   After the withdrawal took effect, the U.S. was no longer bound to submit Nationally Determined Contributions (NDCs) every five years, provide accounting of progress, submit biennial transparency reports, or fulfill the general obligation to provide climate finance [^12]
.
*   The executive order directed the US to withdraw from any agreement, pact, accord, or similar commitment made under the UNFCCC [^18] [^19]
.
*   The U.S. retained its right to vote in the Conference of Parties (COP) and its reporting obligations under the UNFCCC [^12]
.

**Impact of US Rejoining (2021):**

*   The U.S. rejoined with significantly increased ambition, committing to reduce greenhouse gas emissions by 50-52% below 2005 levels by 2030 [^20] [^21] [^22]
.
*   It reaffirmed its goal to reach net-zero emissions by 2050 [^20] [^23] [^22]
.
*   In December 2024, the US communicated an economy-wide target of reducing net greenhouse gas emissions by 61-66 percent below 2005 levels in 2035 [^24] [^22]
.
*   This increased ambition positively impacted the overall global emissions targets [^24] [^22]
.
*   The Biden administration relaunched the Global Methane Pledge at COP26, committing to cut total methane emissions by at least 30% by 2030 [^21]
. As of December 2023, 155 countries had signed onto the pledge [^21]
. The US anticipates methane reductions of at least 35 percent from 2005 levels in 2035 [^24] [^22]
.
*   The Climate Action Tracker assessed the US net-zero target as "Average" and noted areas for improvement, such as including international aviation and shipping, committing to net-zero within its borders without offsets, explaining how the target contributes to limiting warming to 1.5°C, addressing fairness gaps, increasing international climate finance, and stopping support for fossil fuels abroad [^20] [^23]
.

**Impact of US Withdrawal (2025):**

*   This withdrawal is expected to have significant negative consequences for global emissions targets [^13] [^25]
.
*   It makes it significantly more difficult to maintain or enhance global ambitions for emission reduction [^13]
.
*   It creates a leadership vacuum, potentially affecting participation by other nations and local governments [^14] [^13]
.
*   As one of the world's largest emitters, the US's decision to "free-ride" could demotivate other countries from pursuing ambitious climate goals [^13] [^26]
.
*   The executive order aims to stop US climate finance contributions, making it much harder to achieve the global climate finance goal of $1.3 trillion per year by 2035 [^13]
.
*   The withdrawal would take effect one year after notification to the UN Secretary-General, with the earliest effective date being sometime in January 2026 [^13] [^14]
.
*   The US remains a party to the UNFCCC [^14] [^13]
.

### **Economic and Political Consequences for Member and Non-Member Nations**

The US withdrawals have significant economic and political ramifications for both the United States and other nations.

**Stated Reasons for US Withdrawal (2025):**

President Trump cited several reasons for withdrawing the U.S. from the Paris Agreement in 2025 [^9] [^10]
:

*   Potential economic damage [^9]
.
*   Unfair burdens on the United States [^9]
.
*   The executive order, titled "Putting America First in International Environmental Agreements," aimed to prioritize American interests in international environmental policies [^9] [^27]
.
*   This action aligned with a broader initiative to "Make America affordable and energy dominant again" by reviewing regulations that unduly burdened energy production and use [^9] [^28]
.
*   The decision also involved limiting financial contributions to other countries for climate change mitigation [^9] [^28]
.

Specific economic burdens cited included [^29] [^30] [^31] [^32] [^33]
:

*   Unfair economic burden on American workers, businesses, and taxpayers [^30]
.
*   The agreement was believed to force unreasonable burdens on the U.S. economy and endanger American sovereignty [^31]
.
*   Concerns that the agreement disadvantages the United States to the exclusive benefit of other countries [^32]
.
*   The claim that it imposes draconian financial and economic burdens and massive future legal liability on the USA [^32]
.
*   The perception that it handicaps the U.S. economy [^32]
.
*   The idea that it restricts U.S. sovereignty and prevents the country from conducting its domestic economic affairs [^32]
.
*   The obligation for the USA to pay potentially tens of billions of dollars into the Green Climate Fund [^32]
.
*   The concern that it hinders the USA from developing its energy reserves and blocks the development of clean coal [^32]
.
*   Compared to nations like China, the agreement places unfair economic burdens [^33]
.
*   President Trump cited a 2017 study that estimated 2.7 million jobs lost by 2025 and $3 trillion in lost GDP by 2040 [^32]
. However, it was clarified that the study estimated the cost of alternative approaches for meeting deep US emissions targets, not a cost-benefit analysis of the Paris Agreement [^32]
.

Specific examples of unfair burdens cited included [^28] [^34] [^35] [^36] [^37]
:

*   The Paris Agreement imposed an unfair economic burden on the United States [^28] [^34] [^35]
.
*   U.S. pledges made under the agreement were disadvantageous [^34]
.
*   The agreement would undermine the U.S. economy and put the U.S. "at a permanent disadvantage" [^36]
.
*   The United States would not sabotage its industries while other countries, like China, could pollute without consequences [^35]
.
*   The agreement inflicted "severe energy restrictions" on the United States [^37]
.
*   The agreement "punishes" the United States "while imposing no meaningful obligations on the world's leading polluters" [^37]
.
*   Limiting the United States' financial contributions to help other countries mitigate and adapt to climate change globally [^28]
.

**Immediate Global Reaction to US Withdrawal (2020):**

The immediate global reaction to the United States formally withdrawing from the Paris Agreement in November 2020 was overwhelmingly negative [^8] [^38]
. Key aspects of the reaction included:

*   Concerns that the withdrawal would undermine global climate efforts and potentially embolden other nations to scale back commitments [^8]
.
*   Damage to the U.S.'s credibility on the world stage, signaling unpredictability and a retreat from global leadership [^8]
.
*   An abrupt end to U.S. climate finance, including halting contributions to the Green Climate Fund (GCF), which provides vital support to developing nations [^8] [^38]
. This particularly affected poorer nations most vulnerable to climate change [^8]
.
*   The U.S. exit was seen as a setback to global emissions reduction targets [^39]
.
*   Many countries reaffirmed their commitment to the Paris Agreement and climate action [^40]
. The EU, for example, confirmed it would stay the course [^40]
.
*   Cities, states, regions, businesses, investors, and philanthropies within the U.S. and globally committed to continue advancing the objectives of the Paris Agreement [^41]
. For example, governors of several U.S. states formed the U.S. Climate Alliance [^38]
.
*   Scientists emphasized that climate change would continue to worsen regardless of the U.S. decision and that moving away from fossil fuels remains crucial [^39]
.

**Consequences for Member Nations of US Withdrawal (2025):**

The withdrawal of the U.S. from the Paris Agreement in 2025 could have several consequences for member nations [^42] [^43] [^44]
:

*   **Reduced Climate Finance:** The U.S. withdrawal could lead to an abrupt end to climate finance, specifically halting contributions to the Green Climate Fund (GCF) [^42]
. This fund assists developing countries in their efforts to mitigate and adapt to climate change [^42]
. The U.S. had previously pledged $3 billion to the GCF, but only a fraction was honored [^42]
. The lack of financial support could slow global progress on climate adaptation and emissions reduction, particularly affecting vulnerable nations like small island nations and those in Africa [^42] [^43]
.
*   **Damaged International Climate Diplomacy:** The U.S. exit could weaken its position in international climate negotiations, signaling instability to other countries [^42]
. This might embolden other nations to scale back their own commitments [^42]
. Without U.S. leadership, countries such as China, India, and the European Union might assert themselves more forcefully on climate-related issues [^42]
.
*   **Policy Impacts and Emission Reduction Efforts:** The U.S. withdrawal could make it more difficult to maintain or enhance global emission reduction efforts, potentially demotivating other countries [^44]
. The executive order aims to stop U.S. climate finance contributions, making it harder to achieve the new global climate finance goal of $1.3 trillion per year by 2035 [^44]
.
*   **Legal and Reporting Obligations:** After the withdrawal takes effect in January 2026, the U.S. will no longer be bound by its obligations under the Paris Agreement [^44]
. These obligations include submitting Nationally Determined Contributions (NDCs) every five years, accounting for progress toward commitments, submitting biennial transparency reports, and providing climate finance [^44]
.
*   **Impact on Global Temperature Rise:** The U.S. withdrawal could have devastating effects on global temperature rise and associated risks [^43]
.
*   **Loss of Privileges:** The United States will lose its right to vote on decisions within the governing body of the Paris Agreement, to nominate members to institutions serving the Paris Agreement, and to participate in emission trading under the Paris Agreement [^44]
.
*   **Influence on National Commitments:** The withdrawal of the U.S. might influence countries to rethink the setting of their national commitments [^43]
.
*   **Creating a Leadership Vacuum:** The U.S. withdrawal could create a leadership vacuum, as it did during the first Trump administration [^44]
.

**Specific Sectors Affected by US Withdrawal (2025):**

The US withdrawal from the Paris Agreement in 2025 has several potential implications across various sectors [^45] [^46] [^47] [^48]
:

*   **Poorest Countries:** The executive order is aimed at stopping the US climate finance contributions, making it more difficult to achieve the new global climate finance goal of $1.3 trillion per year by 2035, impacting the poorest countries directly [^45]
. It may also degrade the international community's trust in the effectiveness of the process [^45]
.
*   **Clean Energy Sector:** Despite the administration's withdrawal, many US organizations are choosing to redouble their efforts to combat climate change [^46]
. The Inflation Reduction Act (IRA), signed into law in 2022, has accelerated growth in the US clean energy sector, driving substantial investments into renewable energy, electric vehicle infrastructure, and climate resilience projects [^46]
.
*   **Fossil fuel industries:** Increased employment may occur in fossil fuel industries such as coal [^47]
.
*   **Private sector:** A growing number of major corporations and financial institutions are backing away from their climate commitments [^45]
.
*   **Global Climate Efforts:** The absence of US leadership raises critical questions about the future of international climate action and the feasibility of meeting the ambitious target of limiting global warming [^48]
. The US withdrawal introduces volatility and uncertainty around the future of climate leadership and complicates international collaboration, undermining the agreement's momentum and setting a concerning precedent for other nations [^48]
.
*   **US Economy:** The U.S. economy may experience enhanced negative climate effects due both to increased U.S. emissions and potential emissions increases from other countries [^47]
.

### **Effects on International Climate Negotiations and Cooperation**

The US withdrawal from the Paris Agreement in 2025 has several potential effects on international climate negotiations [^49] [^50] [^51]
:

*   **Undermining Global Efforts:** The withdrawal weakens the multilateral system and jeopardizes global efforts to combat climate change [^49]
. It makes it significantly more difficult to maintain and enhance emission reduction efforts across the world [^50]
.
*   **Loss of Influence:** The U.S. could be frozen out of future global climate negotiations, losing influence in shaping international climate policy and hindering efforts to hold major emitters accountable [^49]
. The United States will also lose its right to vote on decisions within the governing body of the Paris Agreement [^50]
.
*   **Troubling Message:** The withdrawal sends a message that the United States is an unreliable partner [^49]
.
*   **Financial Impact:** The executive order is targeted at stopping any U.S. climate finance contributions, making the new global climate finance goal of $1.3 trillion per year by 2035 much harder to achieve [^50]
. This impacts the poorest countries and degrades the international community's trust [^50]
.
*   **Potential Domino Effect:** The U.S. withdrawal could embolden other nations to follow suit, potentially triggering a domino effect and destabilizing the global consensus [^49]
. However, as of April 11, 2025, no other country had withdrawn from the PA after the initial U.S. announcement on January 20, 2025 [^51]
.
*   **Leadership Vacuum:** The withdrawal creates a leadership vacuum [^50]
. During the first Trump administration, other nations, as well as state, local, and business leaders, stepped up to fill this void [^50]
.
*   **Impact on US Commitments:** After the effective withdrawal date in January 2026, the U.S. will no longer be bound by its obligations under the Paris Agreement [^50]
. These obligations include submitting NDCs every five years, accounting for progress toward commitments, submitting biennial transparency reports, and providing climate finance [^50]
.
*   **Continued Participation in UNFCCC:** The U.S. is not withdrawing from the UN Framework Convention on Climate Change (UNFCCC), retaining its right to vote in the Conference of Parties and its reporting obligations under the UNFCCC [^50]
.

### **Long-Term Environmental Impacts**

The potential long-term environmental impacts of the U.S. withdrawal from the Paris Agreement in 2025 are significant [^52] [^53] [^54] [^55] [^56]
:

*   **Damage to international climate diplomacy:** The U.S. withdrawal weakens its position in international climate negotiations and could signal instability to other countries, potentially emboldening them to scale back their own commitments [^52] [^53]
. This introduces volatility and uncertainty around the future of climate leadership [^53]
.
*   **A void in climate leadership:** The absence of the U.S., the world's second-largest emitter of greenhouse gases, creates a leadership vacuum that complicates international collaboration and undermines the agreement's momentum [^52] [^53]
.
*   **Challenges to achieving climate targets:** Without U.S. federal leadership, meeting the Paris Agreement's goals becomes more challenging, requiring enhanced ambition from other nations and leadership from subnational actors [^53]
. The U.S. withdrawal makes it significantly more difficult to maintain or enhance emission reduction efforts across the world [^54]
.
*   **Disruption of climate finance:** The withdrawal leads to an abrupt end to U.S. contributions to the Green Climate Fund (GCF), leaving developing countries without critical resources for mitigating and adapting to climate change [^52] [^55]
. The lack of financial backing slows global progress on climate adaptation and emissions reduction, especially in vulnerable nations [^52]
. The new global climate finance goal of $1.3 trillion per year by 2035 becomes much harder to achieve [^54]
.
*   **Increased emissions:** The U.S. withdrawal could lead to it increasing its own emission space, allowing more room to emit carbon, while other countries would have to cut down on their emissions to reach their goals [^55]
.
*   **Loss of influence in carbon markets:** The U.S. will lose the chance to shape the agreement's carbon markets, which are still under negotiation [^56]
.
*   **Impact on developing countries:** The termination of funding to the Green Climate Fund impacts underdeveloped countries that need aid for their climate change projects [^55]
. A large gap is left in the climate aid that developed countries promised to developing countries [^55]
.
*   **A difficult path to rejoining:** A future U.S. president wishing to rejoin the Paris Agreement would have to submit a new NDC that is more ambitious than the last, which could be challenging given potential rollbacks of environmental protections [^56]
.

### **Executive Summary**

The Paris Climate Agreement, a crucial international treaty aimed at limiting global warming, has experienced significant membership changes, most notably the United States' withdrawals in 2020 and 2025, and its rejoining in 2021 [^7] [^1]
. These changes have profound ramifications across multiple dimensions.

The US withdrawals significantly hinder global progress towards achieving the Paris Agreement's emissions targets by creating leadership vacuums, potentially demotivating other nations, and critically undermining climate finance, particularly through the Green Climate Fund [^12] [^13] [^26] [^44] [^54]
. Conversely, the US rejoining in 2021, accompanied by ambitious new emissions reduction targets (50-52% below 2005 levels by 2030 and net-zero by 2050), injected renewed momentum and ambition into global climate efforts [^20] [^21] [^22] [^24]
.

Economically and politically, the US withdrawals damage its international credibility, disrupt climate diplomacy, and potentially embolden other nations to scale back commitments [^8] [^42] [^49]
. The stated reasons for the 2025 withdrawal centered on perceived economic burdens and unfair treatment compared to other nations [^9] [^30] [^32] [^35]
. The immediate global reaction to the 2020 withdrawal was overwhelmingly negative [^8] [^38]
. Member nations face consequences including reduced climate finance, weakened diplomatic leverage, and potential impacts on their own national commitments [^42] [^43] [^44]
. Specific sectors like clean energy, fossil fuels, and the economies of developing nations are particularly affected [^45] [^46] [^47]
.

Internationally, the US withdrawal weakens the multilateral climate system, reduces its influence in negotiations, and sends a troubling message of unreliability [^49] [^50]
. Long-term environmental impacts include increased global emissions, further challenges in meeting climate targets, and lasting damage to international climate diplomacy and cooperation [^52] [^53] [^54] [^55]
. Rejoining the agreement in the future would require submitting an even more ambitious NDC, potentially complicated by domestic policy rollbacks [^56]

---

### References
[^1]: [wikipedia.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFGrznTY1cwvTPmd_ie6_r5JMicOgzQe3ARUEM0VZDMilTHCSqTswPRbWqHmCCAtLsMnKNLrup16Gj8rHWKTsYJddSJVCgEthxP2TTz8u7gtOv5TxlAMenoN4s8GGjkJNWJ9IZnrlsc2g==)
[^2]: [un.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG40Qot0sA_7Z7OV75D9RIZZXr0UFmzJK0eoZI3nplsoD7odoCJ0RAE6kNeLdjG6K-YdWhL6MtPp4CTqDeFLQUZS_zhEtXNRRyd8G3komsAA7DaJjWsJYLZMGNQGn1jLyeS094Q1CdZS7Oa0LDgEQ==)
[^3]: [wikipedia.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEqAIKN0BRVfKJZn-qwsUfeiJrEYo0bJYDoKLM2fj1wAPVyKxRZR0YcbxLGxa6BHn5vpIyMrQJHpd7tDxf5q_mDh6kLw_FeJ8NDqZasPeywCqscnOMlvLN4ohb3PuzbQhbd1_Bodv1kew==)
[^4]: [un.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFbt-XP3p-qgeHB-NQ3MbllMVg-CiyHyB1Sj59NIP38QGhfNlJIMeuk28UwUmCXNv-By52zX-Kn_c9s38kTry6MaJ3qa7r-vePBAAZKLribydqtdukV1T1uy8u8CsD7sbKKtmNtvloj4g4yxNPkYw==)
[^5]: [wikipedia.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGx6vYYZnRdrJCVAPXLN1N2O32hxRBQ_ldtgjQcfaOr1tXXY4eb2ON2o-UpcGEscHoNiKVKks1Hvo9Eck0XzvRXF92w4Qg6DYYmycbemxJc9Fl3SYOpH2uKR95L2wtFcaGOOqzvTzLAXGzjnqSYoFo7Ea_yGQJFzIuS0OdHYTzA)
[^6]: [wikipedia.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH-82Y18WrQsgydoJ1cX_7uPxQmlw1XrGl_V62zljJRFvkI9AyQOsPglJCc6LINFNDRz--NZd9efHSi2O_4YTU0StXHQER9OYjgcFc6GjKg867BGAURLPvC10gYvDxfS4gauKYmPefKFgb025kBQlzrCcwO3FU1L2i9nGTwSpwg)
[^7]: [cfr.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGtjAB2VoQMaxMDp5lpbGe_KaUoV3RtUCTPeIjayvoClANBpwBo9wsLsG706s_9YnWKmSRoLDagSuPsHI8VmUlv4JUKw1ALyx6A8JD3M09N4xrggAlrpjzIL76c7HcGQNAReMS_T_2OD01_ZE2pan8R5Estx0UkzUPJEPb72UrRUkfm)
[^8]: [time.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHqCTtuRLrvBBjXCh430JHYHmKduOqmqdQ81-8eIjaP1PuHLrXkJBWJf5Ht7tOCxzCPa3uVlfJM9RXn-qnhMO_4766Gk11T5oyw0e_4XcW5WSwdsDdeE-pIrSx7DfnCvZmPzKcZoGnuMiLbeQLByoRxRxokM38mPWcemDW9D69kdxs=)
[^9]: [whitecase.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGyW1vxTYFMC0rpeO8vkv9yeC_H65UzBnE09kCCqk1rUeTfBntUaL3NCPEvFF-aO3zSqzeSOQFoR_DXUw4K_QLmg9TD1zv6OjT5owo4Jo7guvfcUAhBsUJC6E_ecSl2oRkRWDI3UE0uoYcvDBgDmkjmGKhFZmzM1OCjEbWwD9u-BpF8dnBzsCP1RXv4TIFgFshs3Bn76XU=)
[^10]: [congress.gov](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEoYfHM1t64XWx1UR8dq3MMF1H-tuko8Hwok5jm9-fBCRFbZ11awdE-L0qFc8mR3Wzepv1etnFhQaeCTehQ9_gfbUcR5zURa7uRzri230Qqp52wbblE8vyXVgJMQ-DJBV-O1PQ-pBA=)
[^11]: [woodwellclimate.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHvwkuJ0-g6o-LtL109V_5CfAJ7XoL5M6IwHRVB9qZZLYiFc9V_QtoY2K4n7WupZ86uLGEUjYhMMKCU4HeCDT_oPSkHiKJZoCAg2mTP9EU9kJye51A4aAvunzf2Kr_JQUGGjSKqQ_qizPugbePj_LVNHQ2VqhFIDCw=)
[^12]: [woodwellclimate.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHAC4OrdcjWhIrsl6wgHIMrOGQUnJ3txYaKVF0_bUSVHyjw-Vo6k6IrxXr0NT2VMUZHHZ_jeWo5sXJm_FJc1KQOkpfv7jNIWAPgRtDrPaMO1p99kmzsF0weVPa688BlI5j2uPIXf45aF2AaZQkOyfZBFaEp5WhkI2w=)
[^13]: [woodwellclimate.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFRXxv5_-MilnaoFjvXu-zM49RbYjOIYgJKX3o-AVuEoABgZX1-3zlFrzG9hK9TL7kuDXoW2r_PColCORNUH5NScWz3gCzNFVkwKgAqVTeIdiMyILTjGHQPAln-nRH3rjPz2QBqb0MQp4nC42I1xLoowSxYzksjrC5M)
[^14]: [whitecase.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGguty8ieW66HXvblxxWaYXbYbNuli4JXM96QOqmnvhWEQKBnCtxNbQhPuQ5a0ai4RD4OHNNTfwVrlFUHF67tSCDXafk2qskXsdIEtgOutsFX-PbfF5_pn2dLyPYfLj_tHDFknJz_ncU0Gu1hvzd2AdZmnH-eN9KIJNh-4WB1o2tCM6PwPyKH6haCF35ovvdza8A9tZ-ws=)
[^15]: [ibanet.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEbnJIX6MfYBXOz7UrFHBARPVhPaMJuY8Xd8uJUokqpPewBYeBaMHO_dZ-79_ZZ3oVEjRKJfs1M6fuz83nzLDY--Ain7uWRzroOxdEj4yHoMLSLx7NZNXqV4Tmo_QJDrDYm7qc4Yp3nr-o3LuGp5gEihf7H5ryk0KH2jJ_2xsDvEa6bBC4cElWg4ohBniMDXizV8yQt0GQnVQjAp_uNULyDNIvbAw==)
[^16]: [gox-12-e6374.PMC11671060](gs://csr-demo-pubmed/gox-12-e6374.PMC11671060.pdf)
[^17]: [ao2c05089.PMC9774358](gs://csr-demo-pubmed/ao2c05089.PMC9774358.pdf)
[^18]: [whitecase.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG66BFc9GKeKOsu-lJnInVY0UEU97n341od_plYn21JPC7AjKXHL7cUJS-SQSQRAibJW98NLhJ6ZKhP0Oh9LMvQ6aYaGMcGpkQCK8Nz0BPIpE0Dw98RzSQgZZiJoPa1KyOyo7eJw6XsophXC9DmwFJ2vtwIuoKvTYlRR5tDXBXumtR4QDf_h3eLhu7i_goUKZEZiWiczg==)
[^19]: [europa.eu](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGamVyTHkCV2RwpG4vm9nez9kn5Hk89oNDlr47MDPpCm0te2pWq2EcKIDHFSdbIz-oaVblBcMQj0IyOPVhVh5uqoBZQbiB2Hn46jqCGlhptcRNMJ0tXBv2aKIcsPl_SLQlpo0GLg45n2KX-v99XpWw1kI4yFj9UUua3hkVLngbiPCMvCpCng3vSmdG2XH7XZj6Z4Jyi)
[^20]: [climateactiontracker.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFmvohKMwuoWnOpkYZzfgr3CgK5XQs-UD0fcCjwC4a3xiYm4JFvHfKLARDOodcBeVm6sumTMQDDiEqgV4Ju-CGqmaZp5iXlmGve00qMPoJvryfm9NU_wiqBts6Sj31AqNYSEsMcoIPeB_LH)
[^21]: [wri.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE8UdcB91m6Vxs6jZtFSVIK1tDVs9RRp4tZkqMXtgk4ceqq4v_pZypYbZVDqZfBpP-ZKbYU5rTJD5eiC7nNUSYiC8hkca0V17Fqx5TNWy94DuYVVedFDYx3gPRuLQxQ_lIWK2g8097yehQmngP4leS-_EZCwDUGcEiv7Nn8ALF6rPB_VKUNwLkzQv1g8rE=)
[^22]: [archives.gov](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE6HR_k8PeHeCzq9o4f5YyKBLVCM2GRVq-FDK-R1FHnm9dOOnnB45Ptmj_3ne1X2iYYgxb2TqmtDBdeTMcSWVsftEjBEbCxMdsqeJnEuekTPPNl1k36OFCLZ0puPs2JWJrXhxA7LwQdSjGFsxosvlj1Nlg60_LV2KJoY9tm93Rr1LxGYgUqymE2kxQm3VgiaQ7TLvEh2ZD7YnxIkmmMOt2ZpQynUyhCw6Czz9yPuHldWDOztMp2ulTVqZO8ej6wjzalzRQX_3micxCiD-rIRKKltA67MhBRVzURV92vAQdi01iDUWf9DsI77p7UN0snXaYp3WzcWnsKAA40at9K00EPofrT2INTS32ihcyQLc2bU1sBrZOi8Y3B3TipHQ3a7hsHUyvnowKLQuu7KX7x0ouy8pEf8k-Bjw1glpE2ZGl8zZX16qILVbxMT44i2mfOva7asQ==)
[^23]: [climateactiontracker.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG5_S_8bo-Ymi4G6L0SfmV6rE4fYQNxdUGImylnoN08eSLc3Sc9esl3IScyzoKtS_jh1Z0gMNCthVjSP6l-QspLvz6QhMmsHwyLC87K0sohc8sH2XHhsl3I3MkB20sbP9zm7Fq-IJGv8Dd8Rw1XKBBwnYAs9XH0ATpN5fA=)
[^24]: [unfccc.int](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEa8bZiV97_OihgY6pZTSH_ZRpKki5D9_PRrN3mDvztEBcKswrnr32nlNbX27SV2OtABRNdB1_a1TdU6MmgRAxJcr4Q4-4PLsW8eYcT_T0OVKphlopmApiN8FaVM0tV6RawPtqnjERRaDZ65EHzSPCRvPwTaJaSSukv9WuFeG4ym33p5VAu1A8UOt4=)
[^25]: [europa.eu](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEdLlHcMxVi_q0D_0fj9AGNt2s0BKCXP7RGgB7X2dy8NiKcjYtP2Vgz_-R88OY2Qkdl5findJhcuxEEFE5BjYlCZWHs_hwOWBoUsSgmuIJ7uorUNET47L9-Xng_ToxVntGtB4MZIdkObkdbiEay5I6ceYnypQVfSQ0PNFMSwKwXOMhQIckU6yfm4yMn1pKXqMAQsW4kfQ==)
[^26]: [earth.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFbmVE2_BKBj6SxnRrL74_QQn3gd7iFao5wzw4qtMqCYoHweiRNU4PDAIaE_sXOVFVLP_h_5xQdikKE_ZyQ7SrFnEyPXWnacEw8Fgqu57F5lwtzlV2yONYdQntSOqSUHc5r8ssiTF7yObayKYFWZKgj-X7G4zzUdmzY8s7zuU8GjvjYu5SJRMXARsnbXC2NPg7ASEWfQmzODW-WdvIeHMuY2r92z4i0IgkAojVDOSPTxjtFA-F4n8Va2Zpx9nuStEgpV0IVfQk3ggIo2eLOuc90NWBoCS63le8=)
[^27]: [europa.eu](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG7vJAGtWM_8ipRtHU1k3hoJTEXceQ376Zk3dhxSzMJ1umeOevGzur4D1tjwtitX9LxCbrgzloxWKJUILeXC69QjckV7sWYgQQ70QtMQPSuihT0DiVHce_XMqoeBMAmZNaG_0vc1xxjS-gmO4ygIElr6NH_JAkTCol1QToPDZXx3Eg9b-7-kthwDa2Joq4oHZXaDKFIDg==)
[^28]: [whitecase.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGPbzxiG7Y47Yx0hbGmLOFW1f3Wog5qVn5DQDeJvMTT7pZs7Gy1e4sc8WY1RTzJWByd5Y5HvNR0cZFQUrR45WUvsOIjL7KmCILgLFHyDB7ojGZSxV_UsYGOXqxxf25Ym3eCRFAb86j-esChp0SOn-VolVV-Zt29kRrqQ0VYWERuFsvIuWKiHkcyJ0Df6Wz_L8zncec0xuk=)
[^29]: [whitecase.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFFs_zVeNN-I1cb1uR8Oc_p7DSUXNB06Z2b0AyuIkdDPTlOnHM164QLF3G52zTOwa7zoCnnKyk9g2M2taze9UWJcGZUw5ck5SeSgiHoQ7hXpHBvMgvpuDAypOw8WJi3FwjQPaq0mfQoSZypdM2baSfppUpZNMK9-bI-yJGTDUxgxDy9TE2LEPeT7lSAyd38wQWSQzejJw==)
[^30]: [state.gov](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFksVj-N6HSjUWUVolQ2q3UkVez8MY_1v4riPeOliq5PP8qOF0q8JdSOBW_QUUm4NIorRKVRFIeyYIpgiO8sBkOQaJxXqQPFzMThugLSbBCPYDXfrsyWXcR57LOtZ3Hw5GBhRZCX7TEyuJC1vQHvKozoRz5sE0UwLXsYqDbE_L6Lvg3ZAm-)
[^31]: [pitt.edu](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG3uDbkl3sykwgf8Ctf5klMgOqUfJYQq3Jpwa-AWQiNrTVf1b9a4NfU2yV_nglfl-PetOiWrfc171sh6h_yiOHMENcWMvlasYhid1BOSUE1Ief5VdvxN25-76zNiluJFlAo4UFEDubPNu1naPzIt4RZuu3pSCN4k52yeShg3KYQFvYdFD5GWgFxuyEU6UPn2zYU6PFUxDNTigMBQcYnYYItoFg_ozVdQ2pL8H9N1A==)
[^32]: [hpacenter.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG14-YYrcfvFbu0nxWH9BHxeGYcDts__dlog_o1zCh-u1QZwHtu4xXYf6pcbGgayzUumkBux4rjqE94l5mdbce4ixOaCNe8b2KXb9KF4w-SvpoB-YH2V2DV00K8Q_U11nEPr6KtW0LvxF0vswM3PHeSDSGJkY6np7I_JH-bQD8JTnFaT1lqifOCkFxgZ7vbm674LPEN)
[^33]: [osmosisim.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGM6JATzO2B80GSjrRdZeW500546EB64tnX2AjHAmZCNlP81Erps2aauAiZNTwj68DSXZTEhX9CYBy0RjEEwPVRCqeREF6w63-oWU5fCeNR-BbpJA4hU5hNYqDUZLTjsQc7qSuZwJzNrenlT5cTl_ucj17bJsxj8EajE_lbELMawTZrSPcM9V31estPPjN-jZCaPg==)
[^34]: [state.gov](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFrt1sY1xYLVd-lA5svc_VyB48B-UNrIxXYoiUSIJhJU32m0OEiRLz8Uyf5FI5qIL8UzGpm_y6slNi1Kup2i2NuLOxqdL_OziWeogKywLymEdJSBYZhaASdeD0VtmtVyQ7-f7JK9L3luon90amSS4NeqVTc-JmqGfFplcRPvkPY-UxaCcymtg==)
[^35]: [earth.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFzV52DfTJc-SY1cBbXZl2PUd4c3-8OICuqzWhHBzUb0ML2W-4o5FXGqD7s6DfhSk_lixry-9_xe3pOEJOyU365FQiQGU6Rpk1A7Oo-jo_A2Ky0RPB4oAYR1JWMAmSFZMN4Wp51LhZYcGPhLkLQbdeXwvIpQ0fWsj7lFjxPhv1fvfjNhFHNXiAlICwy5AkUcoYAuBdpIcDJut5Vlfm0NSSg9PjYuPsnn581r1j4CLtmlJbyYTUffvA0Kicp6HfQfz_cSZnuAaOi9AwMR1i1uJeegvU73_V2IMk=)
[^36]: [wikipedia.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGTJ5t3r7nqIaJU6vG4s06BqZvjtPuu6Fv7C-Jxr6rgnLoYoT_QftwzoDj2Vh5x6T7E97rtolmPpW25_6OdWywWzIM3oy-W8OXp0_Wm57DNs7YUZo6jgO6cdit4Huo1lXjXZ3lFDcnJDSSKb4GcY0fBPr59craVU7RPv_amVQc=)
[^37]: [ejiltalk.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHnXEaG8Rys7INpnT16yUEhKxGq3FWHzy1XCifSYOAMn1OvkD84NPI-bzgYkl2gaTVRJz2fof6zy0bhUxS3Sl3s1L4dLRT6J84m72mzhZ3-LZreut0brltpKwGyHj0xyYlfzPwdhUIPhrmpbo-bJCvhClFqgcvgMSRLitG6l8BIEatrUQsUn2_wfovJdAoiGvdFtlwd1MEhXrAyU17b)
[^38]: [wikipedia.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFuxofWraorDAcqml-6BkW5U4tQxK0RrCO6b6ITYiUp2QdB-J3r58_exn8LENzd0G7orF7BouD62qavdf8v00NVP3mylWrbMgaLZramgL_FFJNOI3flFo_fwFELXQBNe45ehMXVIh8F-GF7J0go9ithuJvhaN1Wv3NRs0n6UQ==)
[^39]: [sciencemediacentre.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH1bvl_J1wgso-6G3bQxQjepfhj1EHgVszArHKEI1ckBdsnRb4w1-iGvAflIppGIIbI9d4POCkj0dQGUeMaqpFoPu0gtW2mmBjGltIz_X8uU0FAfllaNXaxogA_NuLTmYfHtZZdzejf9XqVnR0JUH0eCX2366IPBeADP652ELNMth4Gxnj_ttCwXbY9l5kNECH-DRo7I19WniyKPGQJkt87RGcmAPBff5rmTvVY-DE1CtgAfLU9LRG4rA==)
[^40]: [europa.eu](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE98B7CZ4WSEKhl4-B8GDEdBsozGfCQx9SanG9RZ4E_ERPlxBnMzMmZYzov5CLx3Kf0KyK_k7OYi38yJtFh5aHMqUji4DLhgXczBrBfw-xdW_EzmpkqVeiOM8deap68wfWeUIN3IBeE5U15Kcb-hlOhtPNCK4wv54QiB5n3i1AYw2XstoFSXpU16AAhPMofwP-tfsBR)
[^41]: [c40.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFryDcppeakh875T8oMeqwTWiL-R-5OblWG6MNPmttN9ftsL6-8IC9DJsgZ1oErWi3xy0efScikI4SAJ1M9J-_iMM5Fy51pl919omjjn0l5r7Z_R90P0LFUH43ekvABGEM1TFRleuGVCJv5zaCnxD-m2UmBcg1ahMOJKcNsrtzvv5IdqiGpR-gSOFBeIbcWiN4_s4UFlR5e0psSpuEwXQ==)
[^42]: [time.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE9IvCPWXCLTdtpv63Rjy5dLcbW7XZ4l13lEXf4O1czNQXmrs1Muo4OaEpvKhqeJmblAQWBAxtzx9Za_6djx2ZfWrehNxOkrAk3-e1e9y5OS1cJ1VQAhmWbBiaS_WS6sKgw2RIfA8FJZbELn27VOcCbd5pflJWUG0gOkK0ZYepGe9TR)
[^43]: [ox.ac.uk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH4yjTnvIlx6-PunBN7YFd12jl9Zwap_iVqMtRx5kqpTYk9NXoXspGbULlBoqKReJ7yP_w7gnhKzZ9_drjHgBvdHPRQ8n_PJx_gRIj3nldM8qE6e_Kz3SXxxP0v16d_pAFmgO7wZJoRPaqULCGWj5yniQmohqTlLFG53TkrDviV3gNXVoe3LytZyM8rI6hYRoHpTLYg_jjthiokxT2zaYlorahVHhoVf9Wnw-VpJw9M)
[^44]: [woodwellclimate.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHVukyLkIo__v9yJVmz2VWhLsmaWy0oDLadSRlgWBV4WJhxLjrLh68wkxxPrjVRnGkxDORVHnB-2VXNvjvfMCvmk08jucJokoToOQLSgOy1hkPBjAXtDPjK0OgNy8pnBEbz9DQhm3ZTn6Tfn8cCnlDBFBXrJuCx4VoF)
[^45]: [woodwellclimate.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHr2niHCO48GBaydwMrelZwcmAkJEvZaqzgQ_ESuKu8-chAS_ZQ8IJHDML4DIvEKRYiFMiZXfBKYOTsmnCJyHkTgqC6g2IXHwH5dvKOZbSSLaG50wB-t_Jyu7qpLaWoMD4Tqc2Noe9_Lq7-wLdgTqkklYyI7NWn6AQ=)
[^46]: [sweep.net](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFxiDoogmQ6VISgi6LDEi2TDnJHpmyyx8V5KddeBHN16hDZtS0079PkepNOSWFZJ2HzN9A7v9CQgF5H-WA7pSeHvPyYHamG_x5C5P-Mq1-U9Le489I3TvQd2DpzPF6qyrYF9DGUV_eBmd2ymX8JSLuNfoPYVV-JIujLBjZnbjTmVAB7BFydMVgHqvZbFEZtfd-1Ck4=)
[^47]: [congress.gov](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFI64UIFyuSUuf2B3-5dc_0-aKHT_0XwrX9Q57MNvVSYgImj8hCCf6aJDcSCkyIx3LExhF_Nis8H4XK7wz2tkMZP1rgPLTOf4poZlQT29tfH2dU3z-GOHnO_NMtq7DdyXNbSRgkvQ==)
[^48]: [carbon-direct.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH9iKER73x_dhMLBvzurI5_7jy-X66nDFZFAwVXHC5-Z9YJZ8larIY5d0P-bDmYRI4cIlNEQ91UyauDI2R5MKuJDpZ3pjusUDWnL-bQBNpipg1rUooJhjGEOM_FkOoCOtCtVUGzy8VsaC_s_NwnmPN2aCmzuyAM63cW9p8-O8qgtcDER9uxsLQUkEDWSps2WnqkZuYvsKSIhvNH4_wQ87g=)
[^49]: [americanprogress.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEVZ-9C2TqmrcaE9Wxt5IGexgHnyYBnb9z3hPdjyc4ajXe6BRqRVV7RWirbGw0xeogftu4c4pjB_F2l4e5WzcCere-bAhz7go3BJbbuGYs9MEYGsTOxtwPKDPeXymyYYrLskDSTcmllWyFBXEfSUEA6OOZWnFOA7GUelKN5Z6yC9SOFoqo_e8NIjeZiIhUZolQ4qTExQUrKKN-lJwqPfw9GklnhaXs=)
[^50]: [woodwellclimate.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG4vGK8IYhLvF0aH7uEjwbr5IyzfChd-lmE7HLH1SWCaOtN4tovggNczPu5xl20zpM3WRpICR7PY8Jqnk9-y3uAK19S_FMmLmx2bKN_vjFoKw_6X1wihlu0DRAM9nMSyTJ6ZUz9vuP-6HWvF-JrqBMyPG1yEVVwXbrg)
[^51]: [congress.gov](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFEv4nXod8Z7XXKo4XCsQv2e8KZTFb9ozdVN7-vhPumGl57B9gsl98g4fFT-AFcp6UW5hQmXPliPDPO2raHZLV28EFL-Tjk0c37y3v7QIOId3k7jLtuN5DAYM92u6vJxpSEuSAg41A=)
[^52]: [time.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFh_EllUHYB97gzBQzGM7nPUv51VofH1oJYtfissX92odRluW9ZYX-YyHX17OI4MMHpojqHGmh3AY5u0i_AhB4BGN-tXbukwqZ8ALktShMfR7ZGJJXowk8dGiQCtxnrDI0852adb3m58fWkawmEzAd7SSjTxnv9EInH3KFIBP9r2VXS)
[^53]: [carbon-direct.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFRXetLPYBT0n1eSHO6nFrX2E6uT6oR3iy0S6NUZM2ytbZe-ekNvNAulIwKngHXNSotIwpW1-YeA8KZNrN7rKrRWkYQAgATN_S6wjsVGdNsYtzR7fl0UBhv8gNquoBSuJf_6V2QRGosG8iiABS5y6KR1aAI9Yo35ASXYzhvjuhI2V6CdbRmYEMLlOU5uRiKPSkxDAKxAiZkEfdPaE3jYhVb)
[^54]: [woodwellclimate.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFDWnfN_yxWZtSGS3WwG_tMYSGk3KuiQrdeAKlU1cleCIXwyud1j11B7WyUzv1mnVNw6Kf4ORHhgdBj_Z0z5Rfdx0iMYi-iP9C537NpOhIflYMjG9f5ozeeAdresbX0A7OrR9NpzKrfvGVn_rOpWGYZiFBpebnRon96)
[^55]: [wikipedia.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGVz6lPgLaU7R6WU5e26JDR7OrNufUMSQmyCi-DoapWOfmoNudLs90-M3u3fxVQSlZSzvnvJTX8rfxKPGyCx7rlyJnrskuDdmDlwwKn1K5BglM5IsfiF6U6fxch9xa2PiA08u2sZ6ups225x2IYRlnQUvQzdLxDS67AHKNhfzw=)
[^56]: [iisd.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHiJJWwdU8hwBdTPj7EvomRd-EbpbveEbB_AjtwWBe-k3YQEQo6clb0oKyLMiVhwFC6cYO1H4PME-u7At3NYE_4VARf1nnCuYwKjdu7dPitAEuOFoTxWCA9rLgBLicoP5qw_mpXyPnHikVmPZOuSWD4D1mAXeuPIk7NFWoAJwpWT3yVTvEU2R48JnwKKY0=)

#### Version 1 : HTML

In [157]:
report_html = format_segments_to_html(report_segments, references)

print("\n--- Final Generated HTML ---")
display(HTML(report_html))



--- Final Generated HTML ---
